# Ray et al 2013 Extract-Transform-Load
**Authorship:**
Adam Klie (last updated: *06/08/2023*)
***
**Description:**
Notebook to extract, transform, and load (ETL) data from the Ray et al (2013) dataset.
***

In [ ]:
# General imports
import os
import sys
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

# EUGENe imports
import eugene as eu
from eugene import preprocess as pp
from eugene import plot as pl
from eugene import settings
settings.dataset_dir = "/cellar/users/aklie/data/eugene/revision/ray13"

# EUGENe packages
import seqdatasets
import seqdata as sd
import seqpro as sp

# Print versions
print(f"Python version: {sys.version}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Xarray version: {xr.__version__}")
print(f"Eugene version: {eu.__version__}")
print(f"SeqDatasets version: {seqdatasets.__version__}")
print(f"SeqData version: {sd.__version__}")

# Download and load in the dataset to a raw `SeqData` object

In [ ]:
# Load in the downloaded data, or download it if it's not there
sdata = sd.open_zarr(os.path.join(settings.dataset_dir, "ray13_norm.zarr"))

In [ ]:
# Load auxiliary data as well
# wget https://hugheslab.ccbr.utoronto.ca/supplementary-data/RNAcompete_eukarya/z_scores.txt.gz -O /cellar/users/aklie/data/eugene/revision/ray13/z_scores.txt.gz
# wget https://hugheslab.ccbr.utoronto.ca/supplementary-data/RNAcompete_eukarya/e_scores.txt.gz -O /cellar/users/aklie/data/eugene/revision/ray13/e_scores.txt.gz
# wget https://hugheslab.ccbr.utoronto.ca/supplementary-data/RNAcompete_eukarya/hg19_motif_hits.tar.gz -O /cellar/users/aklie/data/eugene/revision/ray13/hg19_motif_hits.tar.gz
# tar -xvzf cellar/users/aklie/data/eugene/revision/ray13/hg19_motif_hits.tar.gz -C /cellar/users/aklie/data/eugene/revision/ray13/
# wget https://static-content.springer.com/esm/art%3A10.1038%2Fnbt.3300/MediaObjects/41587_2015_BFnbt3300_MOESM53_ESM.xlsx -O /cellar/users/aklie/data/eugene/revision/ray13/41587_2015_BFnbt3300_MOESM53_ESM.xlsx

In [ ]:
# Get a list of only the target columns
column_vars = pd.Index(sdata.data_vars.keys())
target_mask = column_vars.str.contains("RNCMPT")
target_cols = column_vars[target_mask]
random_idxs = np.random.choice(np.arange(len(target_cols)), size=9, replace=False)
random_cols = target_cols[random_idxs]
len(target_cols)

In [ ]:
# Subset to set type (A -- training or B -- testing)
sdata_setA = sdata.sel(_sequence=sdata["Probe_Set"].compute() == "SetA")
sdata_setB = sdata.sel(_sequence=sdata["Probe_Set"].compute() == "SetB")

# Preprocess the training set

## Preprocess the seqs
- Padded elements of sequences are replaced with a one hot encoded value of 0.25 spanning each base.

In [ ]:
print(f"Max seq len: {np.max(sp.length(sdata['seq'].values))}")

In [ ]:
# Pad sequences to max length with Ns
pp.pad_seqs_sdata(sdata_setA, length=41, seq_var="seq", pad="both", pad_value="N")
pp.pad_seqs_sdata(sdata_setB, length=41, seq_var="seq", pad="both", pad_value="N")

In [ ]:
# One-hot encode sequences and add fill value
pp.ohe_seqs_sdata(sdata_setA, alphabet="RNA", seq_var="seq_padded", fill_value=0.25)
pp.ohe_seqs_sdata(sdata_setB, alphabet="RNA", seq_var="seq_padded", fill_value=0.25)

## Preprocess the targets
- The values of probe intensities are clamped at 99.95% percentile per binding protein to eliminate outliers and balance the data.
- The probe intensities are normalized to a mean of 0 and a standard deviation of 1.

In [ ]:
# Split only those training sequences in SetA into train and validation sets
pp.train_test_random_split(sdata_setA, "_sequence", test_size=0.2)

In [ ]:
# Plot the distribution of the targets
pl.violinplot(
    sdata_setA, 
    vars=random_cols
)
plt.show()

In [ ]:
# Clamp the targets based on percentiles
pp.clamp_targets_sdata(sdata_setA, target_vars=target_cols, percentile=0.9995, train_var="train_val", store_clamp_nums=True)

In [ ]:
# Check the distribution of the clamped targets
sdata_setA[random_cols].to_dataframe().describe()

In [ ]:
# Make sure they match up with stored values
sdata_setA["clamp_nums"][random_idxs]

In [ ]:
# Plot the distribution of the clamped targets
pl.violinplot(
    sdata_setA, 
    vars=random_cols
)
plt.show()

In [ ]:
# Scale the targets have mean 0 and variance 1
scaler = pp.scale_targets_sdata(sdata_setA, target_vars=target_cols, train_var="train_val", return_scaler=True)

In [ ]:
# Check the distribution of the scaled targets, should be approximately normal but not exactly
sdata_setA[random_cols].to_dataframe().describe()

In [ ]:
# Plot the distribution of the scaled targets
pl.violinplot(
    sdata_setA, 
    vars=random_cols
)
plt.show()

# Preprocess the test set
- We need to apply the clamping numbers from the training set to the test set.
- We need to apply the mean and standard deviation from the training set to the test set.

In [ ]:
# Apply the same clamping to the test set
pp.clamp_targets_sdata(sdata_setB, target_vars=target_cols, clamp_nums=sdata_setA["clamp_nums"].to_series())

In [ ]:
# Check the clamping
sdata_setB[random_cols].to_dataframe().describe()

In [ ]:
# Apply the same scaling to the test set
pp.scale_targets_sdata(sdata_setB, target_vars=target_cols, scaler=scaler, suffix=False, return_scaler=False)

In [ ]:
# Check the scaling
sdata_setB[random_cols].to_dataframe().describe()

In [ ]:
# Take subset for testing, only for tests/use_cases/ray13
sdata_setA_sub = sdata_setA.isel(_sequence=slice(100))
sdata_setB_sub = sdata_setB.isel(_sequence=slice(100))

In [ ]:
# Save the processed data
sd.to_zarr(sdata_setA_sub, os.path.join(settings.dataset_dir, "ray13", "norm_setA_sub_ST.zarr"), mode="w")
sd.to_zarr(sdata_setB_sub, os.path.join(settings.dataset_dir, "ray13", "norm_setB_sub_ST.zarr"), mode="w")
sd.to_zarr(sdata_setA, os.path.join(settings.dataset_dir, "ray13", "norm_setA_ST.zarr"), mode="w")
sd.to_zarr(sdata_setB, os.path.join(settings.dataset_dir, "ray13", "norm_setB_ST.zarr"), mode="w")

# Generate multitask ready data
 - With single task training, we can just filter out NaNs and train on the remaining data.
 - We can't do this for multitask training, so we need to generate a separate `SeqData` object where there are no NaNs.

In [ ]:
# Get the columns that you would keep if you removed columns with a certain percentage of missing values
nan_cutoff = 0.01
nan_percents = sdata[target_cols].to_dataframe().isna().sum(axis=0).sort_values(ascending=False)/sdata.dims["_sequence"]
remove_cols = nan_percents[nan_percents > nan_cutoff].index
keep_cols = target_cols.drop(remove_cols)
len(keep_cols)

In [ ]:
# Make a copy of the training data and subset it to only the columns with < nan_cutoff missing values
sdata_setA_MT = sdata_setA.copy()
sdata_setA_MT = sdata_setA_MT.drop(list(remove_cols) + ["_targets", "clamp_nums"])

In [ ]:
# Get rid of any sequences that have missing values in the remaining target columns
keep_rows = np.where(sdata_setA_MT[keep_cols].to_dataframe().isna().sum(axis=1) == 0)[0]
sdata_setA_MT = sdata_setA_MT.isel(_sequence=keep_rows)

In [ ]:
# We also need to remove the columns from the Set B object, but we don't need to remove any rows since we can just ignore those in the evaluation stage
sdata_setB_MT = sdata_setB.copy()
sdata_setB_MT = sdata_setB_MT.drop(list(remove_cols))

In [ ]:
# Double check that the shapes make sense (Set A object has 2 extra columns, one set and one for train/val split. Set B object has 1 extra column, jus the set)
(sdata_setA_MT.dims["_sequence"], len(sdata_setA_MT.data_vars)), (sdata_setB_MT.dims["_sequence"], len(sdata_setB_MT.data_vars))

In [ ]:
# Check if copy worked
(sdata_setA.dims["_sequence"], len(sdata_setA.data_vars)), (sdata_setB.dims["_sequence"], len(sdata_setB.data_vars))

In [ ]:
# Doubke check that there are no missing values in the remaining columns
sdata_setA_MT[keep_cols].to_dataframe().isna().sum().sum()

In [ ]:
# Take subset for testing
sdata_setA_MT_sub = sdata_setA_MT.isel(_sequence=slice(100))
sdata_setB_MT_sub = sdata_setB_MT.isel(_sequence=slice(100))

In [ ]:
# Save the processed data
sd.to_zarr(sdata_setA_MT_sub, os.path.join(settings.dataset_dir, "ray13", "norm_setA_sub_MT.zarr"), mode="w")
sd.to_zarr(sdata_setB_MT_sub, os.path.join(settings.dataset_dir, "ray13", "norm_setB_sub_MT.zarr"), mode="w")
sd.to_zarr(sdata_setA_MT, os.path.join(settings.dataset_dir, "ray13", "norm_setA_MT.zarr"), mode="w")
sd.to_zarr(sdata_setB_MT, os.path.join(settings.dataset_dir, "ray13", "norm_setB_MT.zarr"), mode="w")

# Generating a presence/absence matrix per probe
- We need to generate a presence/absence matrix per probe to use for evaluation
    - This presence/absence matrix is a binary matrix where the rows are all possible k-mers and the columns are probes.
    - The value of a cell is 1 if the k-mer is present in that probe and 0 otherwise.

> **Note**
> Each one of these matrices takes about 15 minutes to generate!

In [ ]:
# Helper function to generate a presence/absence matrix
from ray13_helpers import generate_all_possible_kmers, kmer_in_seqs

In [ ]:
# Generate all possible 7-mers and check
a_probes = pd.Series(sdata_setA["seq"].to_series())
a_probes_MT = pd.Series(sdata_setA_MT["seq"].to_series())
b_probes = pd.Series(sdata_setB["seq"].to_series())
kmers = generate_all_possible_kmers(n=7, alphabet="ACGU")
len(a_probes), len(a_probes_MT), len(b_probes), len(kmers)

In [ ]:
# Generate the Set A presence/absence matrix
a_hits = np.array([a_probes.str.contains(kmer).astype(int).values for i, kmer in tqdm(enumerate(kmers), desc="Searching for kmers in probes", total=len(kmers))])
np.save(os.path.join(eu.settings.dataset_dir, "ray13", "setA_binary_ST"), a_hits)
a_hits.shape, np.all((a_hits == 1).sum(axis=1) >= 155)

In [ ]:
# Generate the Set A presence/absence matrix
a_hits_MT = np.array([a_probes_MT.str.contains(kmer).astype(int).values for i, kmer in tqdm(enumerate(kmers), desc="Searching for kmers in probes", total=len(kmers))])
np.save(os.path.join(eu.settings.dataset_dir, "ray13", "setA_binary_MT"), a_hits)
a_hits_MT.shape, np.all((a_hits_MT == 1).sum(axis=1) >= 155)

In [ ]:
# Generate the Set B presence/absence matrix
b_hits = np.array([b_probes.str.contains(kmer).astype(int).values for i, kmer in tqdm(enumerate(kmers), desc="Searching for kmers in probes", total=len(kmers))])
np.save(os.path.join(eu.settings.dataset_dir, "ray13", "setB_binary"), b_hits)
b_hits.shape,  np.all((b_hits == 1).sum(axis=1) >= 155)

# DONE!

---

# Scratch

In [ ]:
settings.dataset_dir = "/cellar/users/aklie/data/eugene/revision/ray13"

In [ ]:
for zarr in ["norm_setA_sub_MT.zarr", "norm_setB_sub_MT.zarr", "norm_setA_MT.zarr", "norm_setB_MT.zarr"]:
    sdata = sd.open_zarr(os.path.join(settings.dataset_dir, zarr))
    print(zarr, sdata.dims["_sequence"], len(sdata.data_vars))
    if "train_val" in sdata.data_vars:
        print(np.unique(sdata["train_val"].values, return_counts=True))
    else:
        print("No train_val column found")

In [ ]:
for zarr in ["norm_setA_sub_ST.zarr", "norm_setB_sub_ST.zarr", "norm_setA_ST.zarr", "norm_setB_ST.zarr"]:
    sdata = sd.open_zarr(os.path.join(settings.dataset_dir, zarr))
    print(zarr, sdata.dims["_sequence"], len(sdata.data_vars))
    if "train_val" in sdata.data_vars:
        print(np.unique(sdata["train_val"].values, return_counts=True))
    else:
        print("No train_val column found")